In [ ]:
###Import statements
import arcpy
import os
from arcpy import env

###Define the workspace
top_folder = arcpy.env.workspace = r"" #Replace with workspace path
gdb_path = r""
arcpy.env.overwriteOutput = True

#Current project
p = arcpy.mp.ArcGISProject('current')
print('Project', p)

#Current map
m = p.listMaps('')[0] #Replace with Map name

#Join tax parcel shapefile to assessor table
def Join_Parcels_To_Tables():
    for path, dirs, files in os.walk(top_folder):
        for d in dirs:
            print('Base path', path)
            print('Subdirectory', d)
            cur_subpath = os.path.join(path, d)
            print('Current sub-path', cur_subpath)
    
            arcpy.env.workspace = cur_subpath
            fcs = arcpy.ListFeatureClasses()
            print('All subpath FCs', fcs)
            
            for fc in fcs:
                if fc[4:7] == "Tax":
                    parcel_fc = fc
            tbls = arcpy.ListTables()
            for tbl in tbls:
                if tbl[4:7] == "Ass":
                    assessor_tbl = tbl

            x = arcpy.management.AddJoin(parcel_fc,"LOC_ID",assessor_tbl,"LOC_ID")
            outfc = parcel_fc[:4] + "_Parcels"
            arcpy.management.CopyFeatures(x, outfc)

    lyrs = m.listLayers()
    for lyr in lyrs:
        if 'TaxPar' in lyr.name:
            m.removeLayer(lyr)
                        

Join_Parcels_To_Tables()

### Merge parcels into one file
def mergeparcels():
    print("Initialize Parcel Merge")
    parcel_merge = "" #Replace with output file name
    p = arcpy.mp.ArcGISProject('current')
    print('Project', p)
    layers = m.listLayers('*Parcels')
    layerNames = []
    for lyr in layers:
        layerNames.append(lyr.name)

    # Merging in one function will exceed the size limit and throw an error;
    # batching list into chunks of 6
    n = 6
    
    # Using list comprehension to create chunks
    batch = [layerNames[i:i + n] for i in range(0, len(layerNames), n)]
    print(batch) 
        
    #print(all_layers)
    #filtered_list = [filt for filt in layers if 'Parcels' in filt]
    i = 0
    for el in batch:
        arcpy.management.Merge(el, parcel_merge + '_' + str(i))
        i += 1

mergeparcels()

def finalmerge():
    parcel_merge_f = "" #Replace with final output file name
    p = arcpy.mp.ArcGISProject('current')
    print('Project', p)
    layers = m.listLayers('*') #Replace with parcel_merge string followed by asterisk
    arcpy.management.Merge(layers, parcel_merge_f)

finalmerge()